In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler, select_urls
from IPython.display import display
from IPython.core.debugger import set_trace

In [2]:
crawler = NewsCrawler()

# 1. URL을 모은다

In [3]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [ ]:
collect_result = crawler.collect(); display(*collect_result)

1.37% (12.30 seconds): Collecting URLs... time                

# 2. 해당 URL의 뉴스를 다운로드한다

In [ ]:
download_result = crawler.download(); download_result